# Dataset of CAT usage across time

The phab task for measuring CAT usage asks to have a dashboard. We might as well populate that dashboard with historic data. To do so, we grab a dataset of all CAT revisions (using the edit tags) and grab data points weekly on Mondays from the first edit through the current time. This dataset will then be augmented with weekly updates from the dashboard code.

In [1]:
import os
import json

import datetime as dt

import tabulate

import numpy as np
import pandas as pd

from wmfdata import hive, mariadb

In [2]:
## We store one long dataset, where each weekly snapshot has four rows

dataset_filename = 'datasets/weekly-CAT-measurements.tsv'

## Dataset

Query:

In [3]:
## NOTE: there are some anonmyous CAT edits, so we coalesce actor_user with 0 to make
## it easy to filter those out later.

cat_revisions_query = '''
SELECT cat_edits.rev_id, actor_user, rev_timestamp, rev_page,
    IF(mobile_edits.rev_id IS NOT NULL, 1, 0) AS mobile_edit,
    IF(android_edits.rev_id IS NOT NULL, 1, 0) AS android_edit
FROM (
    SELECT rv.rev_id, coalesce(ac.actor_user, 0) AS actor_user, rev_timestamp, rev_page
    FROM revision rv
    INNER JOIN change_tag ct
    ON rev_id = ct_rev_id
    INNER JOIN change_tag_def ctd
    ON ct_tag_id = ctd_id
    INNER JOIN revision_actor_temp rat
    ON rv.rev_id = rat.revactor_rev
    INNER JOIN actor ac
    ON rat.revactor_actor = ac.actor_id
    WHERE ctd_name IN ("computer-aided-tagging", "computer-aided-tagging-manual")
) AS cat_edits
LEFT JOIN (
    SELECT ct_rev_id AS rev_id
    FROM change_tag
    JOIN change_tag_def
    ON ct_tag_id = ctd_id
    WHERE ctd_name = "mobile edit"
) AS mobile_edits
ON cat_edits.rev_id = mobile_edits.rev_id
LEFT JOIN  (
    SELECT ct_rev_id AS rev_id
    FROM change_tag
    JOIN change_tag_def
    ON ct_tag_id = ctd_id
    WHERE ctd_name = "android app edit"
) AS android_edits
ON cat_edits.rev_id = android_edits.rev_id
'''

Grab data:

In [4]:
cat_revisions = mariadb.run(cat_revisions_query, 'commonswiki')

In [ ]:
cat_revisions.head()

In [ ]:
cat_revisions.loc[cat_revisions['actor_user'] == 0].tail()

In [7]:
cat_revisions.loc[cat_revisions['actor_user'] == 0].count()

rev_id           227
actor_user       227
rev_timestamp    227
rev_page         227
mobile_edit      227
android_edit     227
dtype: int64

In [8]:
100 * cat_revisions.loc[cat_revisions['actor_user'] == 0].count() / len(cat_revisions)

rev_id           0.063622
actor_user       0.063622
rev_timestamp    0.063622
rev_page         0.063622
mobile_edit      0.063622
android_edit     0.063622
dtype: float64

In [9]:
cat_revisions['rev_ts'] = pd.to_datetime(cat_revisions['rev_timestamp'], format='%Y%m%d%H%M%S')
cat_revisions['actor_user'] = cat_revisions['actor_user'].astype(int)

We remove all anonymous edits. Currently we do not know what causes them, and they are few.

In [10]:
cat_revisions = cat_revisions.loc[cat_revisions['actor_user'] > 0]

# Mobile edits

Out of curiosity, how many mobile edits do we have?

In [11]:
len(cat_revisions.loc[cat_revisions['mobile_edit'] == 1])

35616

## Identify first date in the dataset

In [12]:
first_edit = cat_revisions['rev_ts'].min()

Identify the first Monday _after_ that first date.

In [13]:
first_monday = first_edit.date() + dt.timedelta(days = (0 - first_edit.weekday()) % 7)

## Iterate and calculate

Iterate weekly starting from `first_monday` until we're past today's date.

In [14]:
today = dt.date.today()

In [15]:
def count_cat_contribs(df):
    '''
    For the slice `df` of a larger DataFrame with revisions, count the number
    of files and contributors, overall and split by desktop, mobile, and Android.
    
    Returns a `pandas.DataFrame` with four rows and three columns: platform,
    number of contributors, number of files.
    '''

    platforms = ['all', 'desktop', 'mobile', 'android']
    n_contributors = [df['actor_user'].nunique()]
    n_files = [df['rev_page'].nunique()]
    
    ## Desktop edits are non-mobile edits (and Android edits are mobile edits by default)
    n_contributors.append(
        df.loc[df['mobile_edit'] == 0]['actor_user'].nunique()
    )
    n_files.append(
        df.loc[df['mobile_edit'] == 0]['rev_page'].nunique()
    )
    
    ## Mobile edits are non-Android edits
    n_contributors.append(
        df.loc[(df['mobile_edit'] == 1) & (df['android_edit'] == 0)]['actor_user'].nunique()
    )
    n_files.append(
        df.loc[(df['mobile_edit'] == 1) & (df['android_edit'] == 0)]['rev_page'].nunique()
    )

    ## Android edits are Android edits
    n_contributors.append(
        df.loc[df['android_edit'] == 1]['actor_user'].nunique()
    )
    n_files.append(
        df.loc[df['android_edit'] == 1]['rev_page'].nunique()
    )
    
    return(pd.DataFrame({
        'platform' : platforms,
        'n_contributors' : n_contributors,
        'n_files' : n_files
    }))


In [16]:
cur_date = first_monday

results = list()

while cur_date < today:
    cur_ts = dt.datetime.combine(cur_date, dt.time(0, 0, 0))
    
    summary_df = count_cat_contribs(cat_revisions.loc[cat_revisions['rev_ts'] < cur_ts])
    summary_df['snapshot_timestamp'] = cur_ts
    results.append(summary_df)
    
    ## advance one week
    cur_date += dt.timedelta(days = 7)

In [17]:
cat_summary = pd.concat(results)

In [18]:
cat_summary

,platform,n_contributors,n_files,snapshot_timestamp
0,all,2,3,2019-11-11
1,desktop,2,3,2019-11-11
2,mobile,0,0,2019-11-11
3,android,0,0,2019-11-11
0,all,16,440,2019-11-18
...,...,...,...,...
3,android,6,267,2020-05-18
0,all,3459,104617,2020-05-25
1,desktop,3180,96183,2020-05-25
2,mobile,481,8202,2020-05-25


## Write out datasets

We add the `snapshot_method` column and set it to `summary` for data added using our "summary statistics" approach. For fresh data gathered weekly, it will be `live` instead, as it'll reflect the status of the live replica at that point in time. This allows us to separate between data points that are based on estimates of available data ("summary", where edits to deleted files aren't counted) and those that reflect a snapshot of the database when the measurements were gathered ("live").

In [19]:
cat_summary['snapshot_method'] = 'summary'

In [20]:
output_columns = ['snapshot_timestamp', 'snapshot_method', 'platform', 'n_contributors', 'n_files']

cat_summary.to_csv(dataset_filename, columns = output_columns,
                    header = True, index = False, sep = '\t')